In [1]:
import pandas as pd
import numpy as np  
import random

from sklearn.metrics.pairwise import cosine_similarity

In [2]:
team = input('Welcome to Dota Captain\'s draft, kindly input your team id:')
side = input('Which side are you playing for?\n A. Radiant \tB. Dire')
ban_first = input('Do you wish to take first ban?\n A. Yes \tB. No')

In [3]:
drafts = pd.read_csv('./data/sparse_matrix.csv', index_col=0)


max_index = drafts.index.max()
new_index = max_index + 1
new_row = pd.DataFrame([[0] * drafts.shape[1]], columns=drafts.columns, index=[new_index])


# Set user team and a random opposition
user_team = float(team)
radiant_teams = drafts['radiant_team_id'].unique()
dire_teams = drafts['dire_team_id'].unique()
all_teams = list(set(radiant_teams) | set(dire_teams))
filtered_teams = [t for t in all_teams if t != user_team]

random_team = random.choice(filtered_teams)


ancient1 = 'radiant' if side == 'A' else 'dire'
ancient2 = 'radiant' if side == 'B' else 'dire'
new_row.loc[new_index, f'{ancient1}_team_id'] = user_team
new_row.loc[new_index, f'{ancient2}_team_id'] = random_team

drafts = pd.concat([drafts, new_row])

all_drafts = drafts.drop(columns=['radiant_win', 'team'])
all_drafts

,78_1,78_2,78_3,78_4,78_5,78_6,78_7,78_8,78_9,78_10,...,32_11,32_12,32_13,32_14,32_15,32_16,32_17,32_18,radiant_team_id,dire_team_id
7750912161,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9425660.0,8629317.0
7750914469,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8629005.0,8629318.0
7750915644,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8961813.0,9330489.0
7750937564,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9344594.0,9395679.0
7750968496,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9330489.0,8961813.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7881664207,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8957156.0,8970060.0
7881677439,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9255039.0,2163.0
7881696382,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8849990.0,8936613.0
7881723710,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8936613.0,8849990.0


In [4]:
# Retrieve one opposition draft

opposition_banfirst = (ban_first != 'A')
rad_ban = 0.0 if not opposition_banfirst else 1.0
dire_ban = 1.0 if not opposition_banfirst else 0.0

# Filter drafts DataFrame based on the conditions
all_opposition_drafts = drafts[
    ((drafts['radiant_team_id'] == random_team) & (drafts['team'] == rad_ban)) |
    ((drafts['dire_team_id'] == random_team) & (drafts['team'] == dire_ban))
]
all_opposition_drafts
opposition_draft = all_opposition_drafts.sample(n=1)
opposition_draft.drop(columns=['radiant_win', 'radiant_team_id', 'dire_team_id', 'team'], inplace=True)

columns_with_value_1 = opposition_draft.columns[(opposition_draft == 1).any()]
print(columns_with_value_1)
opp_indices = [index.split('_')[0] for index in sorted(columns_with_value_1, key=lambda x: int(x.split('_')[1]), reverse=False)]
print(opp_indices)

Index(['78_5', '95_14', '63_8', '61_15', '70_9', '110_10', '120_4', '19_7',
       '86_12', '55_13', '11_4', '123_6', '81_16', '114_2', '128_2', '49_11',
       '69_12', '18_17', '5_11', '109_15', '106_1', '56_18', '64_8', '16_3'],
      dtype='object')
['106', '114', '128', '16', '120', '11', '78', '123', '19', '63', '64', '70', '110', '49', '5', '86', '69', '55', '95', '61', '109', '81', '18', '56']


In [11]:
def cosine_similarity(vec1, vec2):
    vec1 = np.array(vec1)
    vec2 = np.array(vec2)
    dot_product = np.dot(vec1, vec2)
    magnitude_vec1 = np.linalg.norm(vec1)
    magnitude_vec2 = np.linalg.norm(vec2)
    if magnitude_vec1 == 0 or magnitude_vec2 == 0:
        return 0.0
    similarity = dot_product / (magnitude_vec1 * magnitude_vec2)
    return similarity

def rank_similarities(df):
    # Ensure the DataFrame is numerical
    df = df.apply(pd.to_numeric, errors='coerce').fillna(0)
    
    last_entry = df.iloc[-1].values
    other_entries = df.iloc[:-1]
    
    # Compute cosine similarities
    similarities = []
    for idx, row in other_entries.iterrows():
        similarity = cosine_similarity(last_entry, row.values)
        similarities.append((idx, similarity))
    
    # Create a DataFrame for the results with original indices
    similarity_df = pd.DataFrame(similarities, columns=['Index', 'Cosine_Similarity'])
    
    # Sort the results in descending order
    ranked_df = similarity_df.sort_values(by='Cosine_Similarity', ascending=False)
    
    return ranked_df

In [15]:
first_team_bans = [1, 4, 7, 10, 11, 19, 22]
second_team_bans = [2, 3, 5, 6, 12, 20, 21]
first_team_picks = [8, 14, 15, 18, 23] 
second_team_picks = [9, 13, 16, 17, 24]

first_ban = [1, 4, 7, 10, 11, 19, 22 , 8, 14, 15, 18, 23]
second_ban= [2, 3, 5, 6, 12, 20, 21, 9, 13, 16, 17, 24]
optimized_ban_picks = [1, 2, 2, 3, 4, 4, 5, 6, 7, 8 , 8, 9, 10, 11, 11, 12, 12, 13 ,14, 15, 15, 16, 17, 18][::-1]

def print_similar(similar_drafts, order):
    top_n_similar = similar_drafts.sort_values(by='Cosine_Similarity', ascending=False)
    top_n_similar = top_n_similar.head(5)

    for index in top_n_similar.index:
        # Get the row from the drafts DataFrame corresponding to the current index
        similar_row = all_drafts.loc[index]
        
        # Filter columns in the row that match the pattern '_order'
        matching_columns = similar_row.filter(like=f'_{order}')
        
        # Find the matching columns that have a value of 1
        matching_column_names = matching_columns[matching_columns == 1].index.tolist()
        
        # Print the matching values if any
        if matching_column_names:
            cleaned_column_names = [name.split('_')[0] for name in matching_column_names]
            
            print(f'Suggested Heroes: {cleaned_column_names}')
            
for i in range(1, 25):
    optimized_order = optimized_ban_picks.pop()
    if i in first_ban:
        if ban_first:
            if i!=1:
                print_similar(similar_drafts, optimized_order)
            value = input('Input hero id:')
        else:
            value = int(opp_indices[i-1])
        # value = 1 if ban_first else int(opp_indices[i-1])
    elif i in second_ban:
        if ban_first:
            value = int(opp_indices[i-1])
        else:
            print_similar(similar_drafts, optimized_order)
            value = input('Input hero id:')
        # value = int(opp_indices[i-1]) if ban_first else 1
    else:
        continue
    column_name = f'{value}_{optimized_order}'
    all_drafts.iloc[-1, all_drafts.columns.get_loc(column_name)] = 1
    similar_drafts = rank_similarities(all_drafts)
    
    

KeyError: '_3'

In [8]:

last_row = all_drafts.iloc[-1]
columns_with_one = last_row[last_row == 1].index
print(columns_with_one)

['2', 114, 128, '3', 120, 11, '5', '8', 19, '20', '25', 70, 110, '69', '80', 86, 69, '75', '73', 61, 109, '5', '28', 56]
Index(['61_15', '2_1', '20_8', '70_9', '110_10', '120_4', '19_7', '86_12',
       '11_4', '28_17', '3_3', '25_8', '8_6', '80_11', '114_2', '128_2',
       '75_13', '73_14', '69_11', '69_12', '5_5', '5_16', '109_15', '56_18'],
      dtype='object')
